In [1]:
import openai
import csv

In [ ]:
def load_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

api_key = load_api_key('api_key.txt')

In [ ]:
# Define functions for input & output

# For getting the chat GPT output
def get_completion(prompt, model="gpt-4", temperature=0.7, max_tokens=5000, top_p=1.0, frequency_penalty=0, presence_penalty=0):
  try:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model = model,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p = top_p,
        frequency_penalty = frequency_penalty,
        presence_penalty = presence_penalty
    )
    if response.choices:
      return response
    else:
      return "No response from the model."
  except Exception as e:
    return f"Error: {str(e)}"

def get_answer(response):
    return response.choices[0].message.content

# For loading the input text and formatting it
def load_file(file_path):
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader) # skip header
        rows_lst = list(csv_reader)
    return rows_lst

def lst2str(lst):
    res_str = ""
    for i in lst:
        res_str += ','.join(i) + '\n'
    return res_str